In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

data = pd.read_csv("MinMax_Scaler_result.csv",encoding="cp949")
data.head()

,CUSTNO,GNO,CBSCORE,CBSCOREGRD,CREDITOTAMT,YSALEAMT,ESTMM,ASSETAMT,IMSAAMT,IMJUAMT,...,KIND_JOB_K,KIND_JOB_L,KIND_JOB_M,KIND_JOB_N,KIND_JOB_O,KIND_JOB_P,KIND_JOB_Q,KIND_JOB_R,KIND_JOB_S,KIND_JOB_T
0,475821,l180202101898,0.949290,0.000000,0.002338,0.223956,0.058085,0.000768,0.004153,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,74417,l230201700120,0.897566,0.166667,0.002221,0.129121,0.014129,0.001920,0.000000,0.015489,...,0,0,0,0,0,0,0,0,0,0
2,387787,l110201603233,0.909736,0.000000,0.000000,0.083846,0.290424,0.000768,0.004153,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,395418,l230201700254,0.884381,0.166667,0.001637,0.079560,0.017268,0.000768,0.004153,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,190372,l200201601418,0.726166,0.833333,0.000818,0.036813,0.262166,0.006143,0.000831,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [2]:
data.columns

Index(['CUSTNO', 'GNO', 'CBSCORE', 'CBSCOREGRD', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM', 'ASSETAMT', 'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'BIZNML_x', 'LABORCNT', 'ACTCD', 'KOSPI', '환율',
       'GDP', '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률', '소비자심리지수',
       'KIND_JOB', 'KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T'],
      dtype='object')

In [3]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T']]
act = data['ACTCD']

In [4]:
from sklearn.model_selection import train_test_split

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 39) (11736, 39) (27381,) (11736,)


In [5]:
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
from imblearn.combine import SMOTEENN, SMOTETomek

#언더 샘플링
enn = EditedNearestNeighbours(kind_sel="all", n_neighbors=10)
tomekl = TomekLinks()

#오버 샘플링
bsmote = BorderlineSMOTE(random_state=42)
adasyn = ADASYN(random_state=42)

#혼합 샘플링
smotee = SMOTEENN(random_state=42)
smoteT = SMOTETomek(random_state=42)


X_under1_train, Y_under1_train = enn.fit_resample(train_x, train_y)
X_under2_train, Y_under2_train = tomekl.fit_resample(train_x, train_y)

X_over1_train, Y_over1_train = bsmote.fit_resample(train_x,train_y)
X_over2_train, Y_over2_train = adasyn.fit_resample(train_x,train_y)

X_comb1_train, Y_comb1_train = smotee.fit_resample(train_x, train_y)
X_comb2_train, Y_comb2_train = smoteT.fit_resample(train_x, train_y)

# 오버샘플링

1. BorderlineSMOTE

In [6]:
def cut_off(y, threshold) :
    Y = y.copy()  # 대문자 Y를 새로운 변수로 하여 기존의 y값에 영향이 가지 않도록 한다.
    Y[Y>threshold] = 1
    Y[Y<threshold] = 0
    return Y.astype(int)

from sklearn.metrics import confusion_matrix
# confusion matrix accuracy(정확도) 계산함수

def acc(cfmat):
    return (cfmat[0,0] + cfmat[1,1])/(cfmat[0,0] + cfmat[1,1] + cfmat[0,1] + cfmat[1,0])

In [7]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [8]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)

In [9]:
from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X_over1_train, Y_over1_train, cv=5).mean()

0.8527828008840668

In [10]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier()

DecisionTreeClassifier()

In [11]:
AdaBoostClassifier()

AdaBoostClassifier()

In [12]:
params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

In [13]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_over1_train, Y_over1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.55,
 'n_estimators': 27}

In [14]:
grid.best_estimator_.feature_importances_

array([0.04168515, 0.04310587, 0.03001219, 0.05683912, 0.09899768,
       0.12072517, 0.03808457, 0.06901981, 0.07845472, 0.17467918,
       0.01441864, 0.00616519, 0.02414338, 0.01587109, 0.03735509,
       0.03675927, 0.00991319, 0.01979243, 0.01789226, 0.        ,
       0.        , 0.01850489, 0.        , 0.        , 0.01514737,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01025445, 0.        , 0.        , 0.        , 0.02217929,
       0.        , 0.        , 0.        , 0.        ])

In [15]:
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score

dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.01, n_estimators = 300)
model = abc.fit(X_over1_train, Y_over1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [16]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[7909, 2757],
       [ 506,  564]], dtype=int64)

In [17]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.94      0.74      0.83     10666
        정상보증       0.17      0.53      0.26      1070

    accuracy                           0.72     11736
   macro avg       0.55      0.63      0.54     11736
weighted avg       0.87      0.72      0.78     11736



In [18]:
accuracy = acc(cfmat)
accuracy

0.7219665985003408

In [19]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.16982836495031617
recall:  0.5271028037383177
f1 score:  0.256889091323161
accuracy:  0.6343089492158681


2. ADASYN

In [20]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_over2_train, Y_over2_train, cv=5).mean()

0.8547931382441979

In [21]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_over2_train, Y_over2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 27}

In [22]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 7)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 27)
model = abc.fit(X_over2_train, Y_over2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [23]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[8274, 2392],
       [ 594,  476]], dtype=int64)

In [24]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.93      0.78      0.85     10666
        정상보증       0.17      0.44      0.24      1070

    accuracy                           0.75     11736
   macro avg       0.55      0.61      0.54     11736
weighted avg       0.86      0.75      0.79     11736



In [25]:
accuracy = acc(cfmat)
accuracy

0.7455691888207225

In [26]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.16596931659693165
recall:  0.44485981308411215
f1 score:  0.24174707973590656
accuracy:  0.6102978982915404


# 언더샘플링

1. EditedNearestNeighbours

In [27]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_under1_train, Y_under1_train, cv=5).mean()

0.8725706459335856

In [28]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_under1_train, Y_under1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.55,
 'n_estimators': 27}

In [29]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.45, n_estimators = 24)
model = abc.fit(X_under1_train, Y_under1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [30]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[9811,  855],
       [ 756,  314]], dtype=int64)

In [31]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.93      0.92      0.92     10666
        정상보증       0.27      0.29      0.28      1070

    accuracy                           0.86     11736
   macro avg       0.60      0.61      0.60     11736
weighted avg       0.87      0.86      0.87     11736



In [32]:
accuracy = acc(cfmat)
accuracy

0.8627300613496932

In [33]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.2686056458511548
recall:  0.29345794392523367
f1 score:  0.28048235819562306
accuracy:  0.6066483419232394


2. TomekLinks

In [34]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_under2_train, Y_under2_train, cv=5).mean()

0.9063086744273375

In [35]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_under2_train, Y_under2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 7,
 'learning_rate': 0.55,
 'n_estimators': 24}

In [36]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 25)
model = abc.fit(X_under2_train, Y_under2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [37]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[10666,     0],
       [ 1070,     0]], dtype=int64)

In [38]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.91      1.00      0.95     10666
        정상보증       0.00      0.00      0.00      1070

    accuracy                           0.91     11736
   macro avg       0.45      0.50      0.48     11736
weighted avg       0.83      0.91      0.87     11736



C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
accuracy = acc(cfmat)
accuracy

0.9088275391956373

In [40]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.0
recall:  0.0
f1 score:  0.0
accuracy:  0.5


C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 혼합샘플링

1. SMOTEENN

In [41]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_comb1_train, Y_comb1_train, cv=5).mean()

0.8686753890693067

In [42]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_comb1_train, Y_comb1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 26}

In [58]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.01, n_estimators = 270)
model = abc.fit(X_comb1_train, Y_comb1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [59]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[5514, 5152],
       [ 246,  824]], dtype=int64)

In [45]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.95      0.65      0.77     10666
        정상보증       0.16      0.67      0.26      1070

    accuracy                           0.65     11736
   macro avg       0.56      0.66      0.52     11736
weighted avg       0.88      0.65      0.73     11736



In [46]:
accuracy = acc(cfmat)
accuracy

0.6516700749829584

In [47]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.16089887640449438
recall:  0.6691588785046729
f1 score:  0.2594202898550725
accuracy:  0.659537249115453


2. SMOTETomek

In [48]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_comb2_train, Y_comb2_train, cv=5).mean()

0.8506028964576956

In [49]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_comb2_train, Y_comb2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 26}

In [50]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 27)
model = abc.fit(X_comb1_train, Y_comb1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [51]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[6907, 3759],
       [ 341,  729]], dtype=int64)

In [52]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.95      0.65      0.77     10666
        정상보증       0.16      0.68      0.26      1070

    accuracy                           0.65     11736
   macro avg       0.56      0.66      0.52     11736
weighted avg       0.88      0.65      0.72     11736



In [53]:
accuracy = acc(cfmat)
accuracy

0.6506475800954329

In [54]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.16243315508021391
recall:  0.6813084112149532
f1 score:  0.26232457718603813
accuracy:  0.6644400672238276
